# Fermi-Hubbard Model

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra, BlockArrays
using UnPack
using JLD

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

## Model

Second-order Born approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = U^2 G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = U^2 G_{ij, \downarrow}(t, t') G_{ij, \uparrow}(t, t') G_{ji, \uparrow}(t', t)
$$


$T$-matrix approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = U^2 T_{ij}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = U^2 T_{ij}(t, t') G_{ji, \uparrow}(t', t)
$$

$$
   T_{ij}(t, t') =  G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t') + U \int_{\mathcal{C}}\mathrm{d}s\; G_{ij, \uparrow}(t, s) G_{ij, \downarrow}(t, s) T_{ij}(s, t')
$$

$t<t'$:
$$
    \int_{\mathcal{C}}\mathrm{d}s\; A(t, s) B(s, t') = \int_{t_0}^{t}{\mathrm{d}\bar{t}}  A^>(t, \bar{t}) B^<(\bar{t}, t') + \int_{t}^{t'}{\mathrm{d}\bar{t}}  A^<(t, \bar{t}) B^<(\bar{t}, t') + \int_{t'}^{t_0}{\mathrm{d}\bar{t}}  A^<(t, \bar{t}) B^>(\bar{t}, t')  
$$
$t>t'$:
$$
    \int_{\mathcal{C}}\mathrm{d}s\; A(t, s) B(s, t') = \int_{t_0}^{t'}{\mathrm{d}\bar{t}}  A^>(t, \bar{t}) B^<(\bar{t}, t') + \int_{t'}^{t}{\mathrm{d}\bar{t}}  A^>(t, \bar{t}) B^>(\bar{t}, t') + \int_{t}^{t_0}{\mathrm{d}\bar{t}}  A^<(t, \bar{t}) B^>(\bar{t}, t')  
$$

$$
    T^<_{ij}(t, t') =  G^<_{ij, \uparrow}(t, t') G^<_{ij, \downarrow}(t, t') + U \int_{t_0}^{t}{\mathrm{d}\bar{t}}\; G^>_{ij, \uparrow}(t, s) G^>_{ij, \downarrow}(t, s) T^<_{ij}(s, t') + U \int_{t}^{t'}{\mathrm{d}\bar{t}}\; G^<_{ij, \uparrow}(t, s) G^<_{ij, \downarrow}(t, s) T^<_{ij}(s, t') + U \int_{t'}^{t_0}{\mathrm{d}\bar{t}}\; G^<_{ij, \uparrow}(t, s) G^<_{ij, \downarrow}(t, s) T^>_{ij}(s, t')
$$

$$
    T^>_{ij}(t, t') =  G^>_{ij, \uparrow}(t, t') G^>_{ij, \downarrow}(t, t') + U \int_{t_0}^{t'}{\mathrm{d}\bar{t}}\; G^>_{ij, \uparrow}(t, s) G^>_{ij, \downarrow}(t, s) T^<_{ij}(s, t') + U \int_{t'}^{t}{\mathrm{d}\bar{t}}\; G^>_{ij, \uparrow}(t, s) G^>_{ij, \downarrow}(t, s) T^>_{ij}(s, t') + U \int_{t}^{t_0}{\mathrm{d}\bar{t}}\; G^<_{ij, \uparrow}(t, s) G^<_{ij, \downarrow}(t, s) T^>_{ij}(s, t')
$$

$$
    \dot{T}^<_{ij}(t, t') = \partial_t\left(G^<_{ij, \uparrow}(t, t') G^<_{ij, \downarrow}(t, t')\right) + U G^>_{ij, \uparrow}(t, t) G^>_{ij, \downarrow}(t, t) T^<_{ij}(t, t') - U G^<_{ij, \uparrow}(t, t) G^<_{ij, \downarrow}(t, t) T^<_{ij}(t, t') \\
    + U \int_{t_0}^{t}{\mathrm{d}\bar{t}}\; \partial_t\left(G^>_{ij, \uparrow}(t, t') G^>_{ij, \downarrow}(t, t')\right) T^<_{ij}(s, t') + U \int_{t}^{t'}{\mathrm{d}\bar{t}}\; \partial_t\left(G^<_{ij, \uparrow}(t, t') G^<_{ij, \downarrow}(t, t')\right) T^<_{ij}(s, t') + U \int_{t'}^{t_0}{\mathrm{d}\bar{t}}\; \partial_t\left(G^<_{ij, \uparrow}(t, t') G^<_{ij, \downarrow}(t, t')\right) T^>_{ij}(s, t')
$$

$$
    \dot{T}^>_{ij}(t, t') = \partial_t\left(G^>_{ij, \uparrow}(t, t') G^>_{ij, \downarrow}(t, t')\right) + U G^>_{ij, \uparrow}(t, t) G^>_{ij, \downarrow}(t, t) T^>_{ij}(t, t') - U G^<_{ij, \uparrow}(t, t) G^<_{ij, \downarrow}(t, t) T^>_{ij}(t, t') \\
    + U \int_{t_0}^{t'}{\mathrm{d}\bar{t}}\; \partial_t\left(G^>_{ij, \uparrow}(t, s) G^>_{ij, \downarrow}(t, s)\right) T^<_{ij}(s, t') + U \int_{t'}^{t}{\mathrm{d}\bar{t}}\; \partial_t\left(G^>_{ij, \uparrow}(t, s) G^>_{ij, \downarrow}(t, s)\right) T^<_{ij}(s, t') + U \int_{t}^{t_0}{\mathrm{d}\bar{t}}\; \partial_t\left(G^<_{ij, \uparrow}(t, s) G^<_{ij, \downarrow}(t, s)\right) T^>_{ij}(s, t')
$$

## Solving

In [ ]:
function integrate(x::AbstractVector, y::AbstractVector)
    if isone(length(x))
        return zero(first(y))
    end

    @inbounds retval = (x[2] - x[1]) * (y[1] + y[2])
    @inbounds @fastmath @simd for i = 2:(length(y)-1)
        retval += (x[i+1] - x[i]) * (y[i] + y[i+1])
    end
    return 1 // 2 * retval
end;

In [ ]:
function fixed_point(x0::AbstractArray, F::Function; 
        mixing::Float64=0.04, abstol::Float64=1e-9, maxiter::Int=1000, verbose::Bool=true, norm=x -> LinearAlgebra.norm(x, Inf))
  x_old = copy(x0)

  step = 0
  while step < maxiter
    x = F(x_old)
    res = norm(x - x_old)
    if verbose
      println("step: $step // res: $res")
    end
    if res < abstol
      break
    end
    @. x_old = mixing * x + (1.0 - mixing) * x_old
    step += 1
  end

  if step == maxiter
    println("No convergence reached.") 
#   else
#     println("Converged after $step steps.")
  end

  return x_old
end

In [ ]:
J = 1.
num_sites = 8;

In [ ]:
# 8-site 3D cubic lattice
h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
setblock!(h, diag_block, 1, 1)
setblock!(h, diag_block, 2, 2)
setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

full_h = BlockArray{ComplexF64}(undef_blocks, [8, 8], [8, 8])
setblock!(full_h, h |> Array, 1, 1)
setblock!(full_h, h |> Array, 2, 2)
setblock!(full_h, zeros(ComplexF64, 8, 8), 1, 2)
setblock!(full_h, zeros(ComplexF64, 8, 8), 2, 1)

full_h = full_h |> Array;

In [ ]:
Base.@kwdef struct FermiHubbardModel
    U::Float64
    
    # 8-site 3D cubic lattice
    H = t -> J .* full_h
end

In [ ]:
struct FermiHubbardData{T}
    GL::T
    GG::T
    FL::T
    FG::T
    
    TL::T
    TG::T    
    
    ΣNCA_c_L::T
    ΣNCA_c_G::T
    ΣNCA_f_L::T
    ΣNCA_f_G::T
    
    # Initialize problem
    function FermiHubbardData(GL::T, GG::T, FL::T, FG::T, TL::T, TG::T) where T
        new{T}(GL, GG, FL, FG, zero(TL), zero(TG), zero(GL), zero(GG), zero(FL), zero(FG))
    end
end

In [ ]:
function fv!(model, data, out, times, t, t′)
    @unpack GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G = data
    @unpack H, U = model 
    
    # real-time collision integral
    ∫dt(i, j, A, B) = sign(j - i) * integrate(times[min(i, j):max(i, j)],[A[t, s] * B[s, t′] for s = min(i, j):max(i, j)])
    
    ΣHF_c(t, t′) = 1.0im * U * Diagonal([FL[k, k, t, t] for k in 1:num_sites])
    ΣHF_f(t, t′) = 1.0im * U * Diagonal([GL[k, k, t, t] for k in 1:num_sites])
    
    out[1] = -1.0im * ((H((times[t] + times[t′])/2)[1:num_sites, 1:num_sites] + ΣHF_c(t, t′)) * GL[t, t′] + 
            ∫dt(1, t, ΣNCA_c_G, GL) + ∫dt(t, t′, ΣNCA_c_L, GL) - ∫dt(1, t′, ΣNCA_c_L, GG)
        )

    out[2] = -1.0im * ((H((times[t] + times[t′])/2)[1:num_sites, 1:num_sites] + ΣHF_c(t, t′)) * GG[t, t′] + 
            ∫dt(t′, t, ΣNCA_c_G, GG) - ∫dt(1, t, ΣNCA_c_L, GG) + ∫dt(1, t′, ΣNCA_c_G, GL)
        )

    out[3] = -1.0im * ((H((times[t] + times[t′])/2)[1 + num_sites:2 * num_sites, 1 + num_sites:2 * num_sites] + ΣHF_f(t, t′)) * FL[t, t′] + 
            ∫dt(1, t, ΣNCA_f_G, FL) + ∫dt(t, t′, ΣNCA_f_L, FL) - ∫dt(1, t′, ΣNCA_f_L, FG)
        )

    out[4] = -1.0im * ((H((times[t] + times[t′])/2)[1 + num_sites:2 * num_sites, 1 + num_sites:2 * num_sites] + ΣHF_f(t, t′)) * FG[t, t′] +
            ∫dt(t′, t, ΣNCA_f_G, FG) - ∫dt(1, t, ΣNCA_f_L, FG) + ∫dt(1, t′, ΣNCA_f_G, FL)
        )    
end

function fd!(model, data, out, times, t, t′)
    fv!(model, data, out, times, t, t)
    out[1:4] .-= adjoint.(out[1:4])
end

In [ ]:
function second_Born!(model, data, times, t, t′)
    @unpack GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G = data
    @unpack U = model
        
    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)
    end
    
    TL[t, t′] = -1.0im .* GL[t, t′] .* FL[t, t′]
    TG[t, t′] = -1.0im .* GG[t, t′] .* FG[t, t′]        
    
    ΣNCA_c_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_f_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(GG[t′, t])
    
    ΣNCA_c_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(FL[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
function T_matrix!(model, data, times, t, t′)
    @unpack GL, GG, FL, FG, TL, TG, ΣNCA_c_L, ΣNCA_c_G, ΣNCA_f_L, ΣNCA_f_G = data
    @unpack U = model
        
    # real-time collision integral
    ∫dt(i, j, A::Function, B::GreenFunction) = sign(j - i) * integrate(times[min(i, j):max(i, j)], [A(t, s) * B[s, t′] for s = min(i, j):max(i, j)])    
    ∫dt(i, j, A::GreenFunction, B::Function) = sign(j - i) * integrate(times[min(i, j):max(i, j)], [A[t, s] * B(s, t′) for s = min(i, j):max(i, j)]) 

    if (n = size(GL, 3)) > size(ΣNCA_c_L, 3)
        resize!(ΣNCA_c_L, n)
        resize!(ΣNCA_c_G, n)
        resize!(ΣNCA_f_L, n)
        resize!(ΣNCA_f_G, n)
    end
    
    ΦL(t, t′) = -1.0im .* GL[t, t′] .* FL[t, t′]
    ΦG(t, t′) = -1.0im .* GG[t, t′] .* FG[t, t′]
    
    x₀ = [ΦL(t, t′), ΦG(t, t′)]
    
    if t == t′
        
        function Fd!(x, i)
            TL[t, t′] = x[1]
            TG[t, t′] = x[2]
            
            integrals = ∫dt(1, i, TG, ΦL) - ∫dt(1, i, ΦL, TG) + ∫dt(1, i, ΦG, TL) - ∫dt(1, i, TL, ΦG)

            return [ΦL(t, t′) - 0.5U .* integrals, ΦG(t, t′) - 0.5U .* integrals]
        end
        TL[t, t′], TG[t, t′] = fixed_point(x₀, x -> Fd!(x, t); mixing=0.5, verbose=false)
        
    elseif t > t′
        
        function Fv!(x, i)
            TL[t, t′] = x[1]
            TG[t, t′] = x[2]
            
            return [
                    ΦL(t, t′) + U .* (∫dt(1, t′, ΦL, TG) - ∫dt(1, t′, ΦL, TL) - ∫dt(1, i, ΦG, TL) + ∫dt(1, i, ΦL, TL)), 
                    ΦG(t, t′) + U .* (∫dt(1, t′, ΦG, TG) - ∫dt(1, t′, ΦG, TL) - ∫dt(1, i, ΦG, TG) + ∫dt(1, i, ΦL, TG))
                   ]
        end
        TL[t, t′], TG[t, t′] = fixed_point(x₀, x -> Fv!(x, t); mixing=0.5, verbose=false)
    end
        
    ΣNCA_c_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_f_L[t, t′] = 1.0im .* U^2 .* TL[t, t′] .* transpose(GG[t′, t])
    
    ΣNCA_c_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(FL[t′, t])
    ΣNCA_f_G[t, t′] = 1.0im .* U^2 .* TG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
# quantum numbers
dim = num_sites

# Define your Green functions at (t0, t0) – time arguments at the end
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

TL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
TG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# Initial condition
N_c = zeros(num_sites)
N_f = zeros(num_sites)

N_c[1:4] = 0.1 .* [1, 1, 1, 1]
N_f[1:4] = 0.1 .* [1, 1, 1, 1]

N_c[5:8] = 0.0 .* [1, 1, 1, 1]
N_f[5:8] = 0.0 .* [1, 1, 1, 1]

GL[1, 1] = 1.0im * diagm(N_c)
GG[1, 1] = -1.0im * (I - diagm(N_c))
FL[1, 1] = 1.0im * diagm(N_f)
FG[1, 1] = -1.0im * (I - diagm(N_f))

TL[1, 1] = -1.0im .* GL[1, 1] .* FL[1, 1]
TG[1, 1] = -1.0im .* GG[1, 1] .* FG[1, 1]

data = FermiHubbardData(GL, GG, FL, FG, TL, TG)
data_2B = deepcopy(data)

model = FermiHubbardModel(U = 2.)
tmax = 5;

In [ ]:
# atol = 1e-7
# rtol = 1e-5;

atol = 1e-5
rtol = 1e-3;

In [ ]:
# callback = (x...) -> second_Born!(model, data, x...)
callback = (x...) -> T_matrix!(model, data, x...)

In [ ]:
@time sol = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> begin
        println(" t: $(x[2][x[3]])")
        fd!(model, data, x...)
    end,
    [data.GL, data.GG, data.FL, data.FG, data.TL, data.TG],
    (0.0, tmax);
    callback = callback,
    atol = atol,
    rtol = rtol,
    dtini=1e-10
);

In [ ]:
figure(figsize=(3, 2))
plot(sol.t[1:end-1], sol.t |> diff)
xlabel("\$t\$")
tight_layout()

In [ ]:
save("FH_3D_T_matrix_sol_U_"*string(model.U)*"_tmax_"*string(tmax)*"_atol_"*string(atol)*"_rtol_"*string(rtol)*".jld", "solution", sol)

### Second Born approximation

In [ ]:
@time sol_2B = kbsolve!(
    (x...) -> fv!(model, data_2B, x...),
    (x...) -> begin
        println(" t: $(x[2][x[3]])")
        fd!(model, data_2B, x...)
    end,
    [data_2B.GL, data_2B.GG, data_2B.FL, data_2B.FG, data_2B.TL, data_2B.TG],
    (0.0, tmax);
    callback = (x...) -> second_Born!(model, data_2B, x...),
    atol = atol,
    rtol = rtol,
    dtini=1e-10
);

## Example plots

### Load data

In [ ]:
loaded_sol = load("FH_3D_T_matrix_sol_U_"*string(model.U)*"_tmax_"*string(tmax)*"_atol_"*string(atol)*"_rtol_"*string(rtol)*".jld")
GFs = loaded_sol["solution"].u
num_points = (loaded_sol["solution"].t |> size)[1]

In [ ]:
t_scale = J

xpad = 8
ypad = 5;

In [ ]:
figure(figsize = (8, 3))

ax = subplot(121)
idx_1 = 1

plot(t_scale .* loaded_sol["solution"].t, [imag(GFs[1][idx_1, idx_1, k, k] .+ GFs[3][idx_1, idx_1, k, k]) for k = 1:num_points], 
    label = "\$ T\$-matrix", lw=1.5, ls = "-", c = "#438E6A")

plot(t_scale .* sol_2B.t, [imag(sol_2B.u[1][idx_1, idx_1, k, k] .+ sol_2B.u[3][idx_1, idx_1, k, k]) for k = 1:size(sol_2B.t)[1]], 
    label = "Second Born", lw=3, ls = "--", c = "#438E6A", alpha=0.5)

xlim(0, t_scale .* tmax)
ax.set_xticks(0:t_scale .* tmax / 5:t_scale .* tmax)
ylim(0, 0.3)
xlabel("\$J t\$")
ylabel("Charge", labelpad = 8)
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.legend(loc = "best", handlelength = 1.9, frameon = false, borderpad = 0, labelspacing = 0.25)

ax = subplot(122)
plot(t_scale .* loaded_sol["solution"].t, 
    [(sum(GFs[1][i, i, k, k] for i = 1:num_sites) |> imag) .+ (sum(GFs[3][i, i, k, k] for i = 1:num_sites) |> imag) for k = 1:num_points] 
    .- sum(N_c .+ N_f)  .|> abs, ls = "-", c = "k")
xlim(0, t_scale .* tmax)
# ylim(0, 2e-3)
ax.set_xticks(0:t_scale .* tmax / 5:t_scale .* tmax)
ylabel("Charge conservation", labelpad = 16)
xlabel("\$J t\$")
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.yaxis.set_label_position("right")

tight_layout(pad = 0.1, w_pad = 0.5, h_pad = 0)
# savefig("fermi_hubbard_T_matrix_T.pdf")

## Testing

In [ ]:
using FFTW, Interpolations

In [ ]:
idx = 1
ρτ, (τs, ts) = wigner_transform_itp(GL[idx, idx, :, :] - GG[idx, idx, :, :], sol.t, fourier=false);
# ρω, (ωs, ts) = wigner_transform_itp(GL[idx, idx, :, :] - GG[idx, idx, :, :], sol.t, fourier=true);

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

Y, X = meshgrid(loaded_sol["solution"].t, loaded_sol["solution"].t);

In [ ]:
cmap = "gist_heat";

In [ ]:
figure(figsize=(7, 3))
t_scale = 1
vmin = -1.0
vmax = 1.0

center = floor(length(ts) / 2) |> Int

ax = subplot(121)
plot(t_scale * τs, ρτ[:, center] |> imag, ls="-", c="C0", lw=1.5)
ax.set_xlabel("\$J \\tau\$")
ax.set_xlim(-t_scale * tmax / 2, t_scale * tmax / 2)
# ax.set_ylim(-1, 1)
ax.set_xticks(t_scale .* [-tmax/2, -tmax/4, 0, tmax/4, tmax/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$  A_{11, \\uparrow}(T_{\\mathrm{max}}/2, \\tau)_W \$")

ax = subplot(122)
heatmap = ax.pcolormesh(X, Y, imag(GFs[1][1, 1, :, :]) .- imag(GFs[2][1, 1, :, :]), cmap=cmap, rasterized=true, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
ax.set_ylabel("\$J t'\$")
ax.set_xlim(0, t_scale * tmax)
ax.set_ylim(0, t_scale * tmax)
ax.set_xticks(t_scale .* [0, tmax/2, tmax])
ax.set_yticks(t_scale .* [0, tmax/2, tmax])
# ax.set_yticklabels([])
tight_layout(pad=0.75, w_pad=0.5, h_pad=0)

# savefig("fermi_hubbard_example_two_times.pdf")

In [ ]:
figure(figsize = (8, 2.5))

ax = subplot(121)
plot(t_scale .* loaded_sol["solution"].t, [(sum(GFs[1][i, i, k, k] for i = 1:num_sites) |> imag) .+ (sum(GFs[3][i, i, k, k] for i = 1:num_sites) |> imag) for k = 1:num_points] 
    .- sum(N_c .+ N_f)  .|> abs, 
    label = "\$ c \$", ls = "-", c = "k")
xlim(0, t_scale .* tmax)
ax.set_xticks(0:t_scale .* tmax / 5:t_scale .* tmax)
ax.set_xticklabels([])
ylabel("Charge conservation", labelpad = 16)
ax.yaxis.set_label_position("right")

ax = subplot(122)
plot(t_scale .* loaded_sol["solution"].t, [imag(GFs[5][idx_1, idx_1, k, k]) for k = 1:num_points], 
    label = "\$ T^<_{11}(t, t)\$", lw=1.5, ls = "-", c = "#438E6A")

plot(t_scale .* sol_2B.t, [imag(-1.0im .* sol_2B.u[1][idx_1, idx_1, k, k] .* sol_2B.u[3][idx_1, idx_1, k, k]) for k = 1:size(sol_2B.t)[1]], 
    label = "\$ \\Phi^<_{11}(t, t)\$", lw=3.5, ls = "--", c = "#438E6A", alpha=0.5)

xlim(0, t_scale .* tmax)
ax.set_xticks(0:t_scale .* tmax / 5:t_scale .* tmax)
ax.set_yticks(1e-2 .* [0, 0.5, 1])
ylim(0, 0.01)
xlabel("\$J t\$")
ylabel("Imaginary Part", labelpad = 16)
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.legend(loc = "best", handlelength = 1.9, frameon = false, borderpad = 0, labelspacing = 0.25)
ax.yaxis.set_label_position("right")


tight_layout(pad = 0.0, w_pad = 0.5, h_pad = 0)
# savefig("fermi_hubbard_example_T_inset.pdf")